In [20]:
import numpy as np
import scipy.io as scio    
import os,re
from scipy import ndimage
from scipy import misc
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import keras
from numpy import random
from keras.models import *
from keras.layers import Input,Flatten, Dense,merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D,Concatenate,Activation
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau,CSVLogger
from keras import backend as keras

In [21]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [22]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

In [23]:
melanoma_filenames = []
others_filenames = []
groundtruth = []

In [28]:
for root, dirnames, melanoma_filenames in os.walk("Dataset/resized_melanoma/"):
    melanoma_filenames.sort(key = natural_keys)
    rootpath = root
    
for root, dirnames, others_filenames in os.walk("Dataset/resized_others/"):
    others_filenames.sort(key = natural_keys)
    rootpath = root
    
for root, dirnames, groundtruth in os.walk("Dataset/resized_gt/"):
    groundtruth.sort(key = natural_keys)
    rootpath = root

In [29]:
print(len(melanoma_filenames))
print(len(others_filenames))
print(len(groundtruth))

374
1626
2000


In [30]:
total_images = np.load('total_images.npy')
ground_truth_images = np.load('gt_labels_binary.npy')

In [32]:
total_filenames = melanoma_filenames + others_filenames
print(len(total_filenames))
total_filenames.sort(key=natural_keys)

2000


In [35]:
segmented_images = np.copy(total_images)
x,y,z = segmented_images[0].shape
for i in range(len(total_images)):
    for j in range(x):
        for k in range(y):
            for l in range(z):
                segmented_images[i][j][k][l] = total_images[i][j][k][l] if ground_truth_images[i][j][k]==1 else 0
    misc.imsave("segmented_images/segmented_"+total_filenames[i],segmented_images[i])

In [39]:
np.save('segmented_images.npy',segmented_images)

In [40]:
np.save('segmented_images.npy',segmented_images)

In [41]:
classification_labels = np.zeros((len(total_images)))
i = 0
for file in total_filenames:
    if os.path.exists("Dataset/resized_melanoma/"+file):
        classification_labels[i]=0
    else:
        classification_labels[i]=1
    i+=1

In [42]:
print(classification_labels)

[1. 1. 0. ... 1. 0. 1.]


In [43]:
np.save('classification_labels.npy',classification_labels)

In [45]:
train_indices = np.random.choice(2000,1600,replace=False)
test_indices = [i for i in range(2000) if i not in train_indices]

In [46]:
print(len(train_indices))
print(len(test_indices))

1600
400


In [47]:
cls_train_images = [segmented_images[i] for i in train_indices]
cls_train_labels = [classification_labels[i] for i in train_indices]

In [48]:
train_mean = np.mean(cls_train_images,axis = (0,1,2,3))
train_std = np.std(cls_train_images,axis = (0,1,2,3))
cls_train_images = (cls_train_images - train_mean)/(train_std+1e-7)

In [49]:
np.save('classification_train_images.npy',cls_train_images)
np.save('classification_train_labels.npy',cls_train_labels)

In [50]:
cls_test_images = [segmented_images[i] for i in test_indices]
cls_test_labels = [classification_labels[i] for i in test_indices]

In [51]:
test_mean = np.mean(cls_test_images,axis = (0,1,2,3))
test_std = np.std(cls_test_images,axis = (0,1,2,3))
cls_test_images = (cls_test_images - test_mean)/(test_std+1e-7)

In [52]:
np.save('classification_test_images.npy',cls_test_images)
np.save('classification_test_labels.npy',cls_test_labels)

In [59]:
train_images = np.load('classification_train_images.npy')
test_images = np.load('classification_test_images.npy')
train_labels = np.load('classification_train_labels.npy')
test_labels =  np.load('classification_test_labels.npy')

In [60]:
def get_unet_model(image_dims):
    inputs = Input((image_dims[0],image_dims[1],image_dims[2]))
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
   

    up6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = Concatenate(axis=3)([conv4,up6])
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = Concatenate(axis=3)([conv3,up7])
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = Concatenate(axis=3)([conv2,up8])
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = Concatenate(axis=3)([conv1,up9])
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  #  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    flatten1 = Flatten()(conv9)
    dense2 = Dense(1,activation='sigmoid')(flatten1)
    
    model = Model(inputs = inputs, outputs = dense2)
    return model

In [61]:
model = get_unet_model((128,192,3))

In [62]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 192, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 192, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 192, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 96, 16)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [63]:
model.compile(optimizer = Adam(lr = 1e-25), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [64]:
lr_reducer = ReduceLROnPlateau(factor=0.5, cooldown=0, patience=6, min_lr=0.5e-6)
csv_logger = CSVLogger('classification.csv')
model_checkpoint = ModelCheckpoint("classification.hdf5",monitor = 'val_loss',verbose = 1,save_best_only=True)

In [ ]:
model.fit(train_images, train_labels, batch_size=10, epochs=20, verbose=1,validation_data=(test_images,test_labels), shuffle=True, callbacks=[lr_reducer,csv_logger,model_checkpoint])

In [66]:
plt.imshow(test_images[5])
plt.show()

In [67]:
test_images = np.expand_dims(test_images,axis=1)

In [68]:
sample_predictions = model.predict(test_images[5].reshape((1,128,192,3)))

In [69]:
predicted_class = sample_predictions[0]>0.5

In [71]:
predicted_class = 0 if predicted_class[0]==True else 1

In [72]:
class_labels = {0:'melanoma',1:'others'}

In [73]:
test_images = np.squeeze(test_images,axis=1)

In [74]:
test_predictions = model.predict(test_images)

In [75]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [76]:
class_names = ['melanoma','others']